In [1]:
import yaml
import torch
from model.gpt import GPT
import tiktoken
from utils.sampling import top_k_logits
from generate import generate_text

with open('config.yaml', 'r') as f:
    config = yaml.load(f, Loader=yaml.SafeLoader)
    

model_cfg = config["model"]
train_cfg = config["training"]
data_cfg  = config["data"]

dataset_path = config["data"]['dataset_path']
vocab_size = model_cfg['vocab_size']
max_T = model_cfg['max_T']
d_model = model_cfg['d_model']
n_heads = model_cfg['n_heads']
d_ff = model_cfg['d_ff']
n_layers = model_cfg['n_layers']
NEG_INF = -1e5


device = "mps" if torch.backends.mps.is_available() else "cpu"


model = GPT(d_model, vocab_size, max_T, n_layers, n_heads, d_ff)

ckpt = torch.load("checkpoint.pt", map_location=device) 
model.load_state_dict(ckpt["model"]) 

model.to(device)
model.eval()

GPT(
  (blocks): ModuleList(
    (0-5): 6 x TransformerBlock(
      (attn): Attention(
        (W_Q): Linear(in_features=512, out_features=512, bias=True)
        (W_K): Linear(in_features=512, out_features=512, bias=True)
        (W_V): Linear(in_features=512, out_features=512, bias=True)
        (W_O): Linear(in_features=512, out_features=512, bias=True)
        (attn_dropout): Dropout(p=0.1, inplace=False)
        (resid_dropout): Dropout(p=0.1, inplace=False)
      )
      (mlp): MLP(
        (fc1): Linear(in_features=512, out_features=2048, bias=True)
        (fc2): Linear(in_features=2048, out_features=512, bias=True)
        (gelu): GELU(approximate='none')
      )
      (ln1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      (ln2): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
    )
  )
  (emb): Embeddings(
    (token_embedding): Embedding(50257, 512)
    (positional_embedding): Embedding(128, 512)
    (embedding_dropout): Dropout(p=0.1, inplace=False)
  )
  

In [2]:
prompt = "Once upon a time"

for t in [0.5, 0.8, 1.0, 1.3]:
    print(f"\n=== temperature = {t} ===")
    print(generate_text(
        model,
        prompt=prompt,
        max_new_tokens=80,
        temperature=t,
        top_k=40,
    ))



=== temperature = 0.5 ===
Once upon a time, a team was in the top of the final.

“I’m going to be a good thing.”

“We’ve got to get a lot of things like,” said W. “I’m not going to do.”

“I’ve been a lot of years,” he

=== temperature = 0.8 ===
Once upon a time, the new company will have to have to be paid against the best, and we will be able to be able to move.

The first time you will get a new version of the final day (and they’d read this will be below), the first time he’s going to win and make it a couple of days to be on the first game.

�

=== temperature = 1.0 ===
Once upon a time.

“If I’m good for a very little bit of thing, a lot of what you are going to feel. I’d like your life.

“The same is a lot of years.”

Hutzan said as much as the most famous of you say that the entire thing is to do, is very efficient, so

=== temperature = 1.3 ===
Once upon a time later, the way we have to ask him. I see it to say that it must come to do.”

It’s a good idea of his work when Trump

In [ ]:
for k in [None, 20, 40, 100]:
    print(f"\n=== top_k = {k} ===")
    print(generate_text(
        model,
        prompt="The future of AI is",
        max_new_tokens=80,
        temperature=1.0,
        top_k=k,
    ))


In [ ]:
def run_with_seed(seed):
    torch.manual_seed(seed)
    return generate_text(
        model,
        prompt="Deep learning models",
        max_new_tokens=60,
        temperature=1.0,
        top_k=40,
    )

print(run_with_seed(42))
print(run_with_seed(42))  # identyczne
print(run_with_seed(123)) # inne


In [ ]:
short = "AI will change the world."
long = "AI " * 200

print(generate_text(model, short, 50, 1.0, 40))
print(generate_text(model, long, 50, 1.0, 40))
